In [ ]:
import scipy.ndimage as sci
import numpy as np
import os
import matplotlib.pyplot as plt
import glob
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from keras.utils import np_utils

In [ ]:
#Load Data and Pre-processing and Visualization
def load_dataSet():
    data = ['forward', 'left', 'right', 'idle']
    X = []
    sizes = []
    for direction in data:
        address = './' + direction + '/*.jpg'
        file_list = glob.glob(address)
        sizes.append(len(file_list))
        print('len of {} is {}'.format(direction, len(file_list)))
        for image in file_list:
            x = Image.open(image)
            x = np.array(x.convert('L'))
            x = x.astype('float32')/255
            X.append(x)
    #labels
    y_forward = 0* np.ones((sizes[0],1))
    y_left    = 1* np.ones((sizes[1],1))
    y_right   = 2* np.ones((sizes[2],1))
    y_idle    = 3* np.ones((sizes[3],1))
    Y = np.concatenate((y_forward, y_left, y_right, y_idle),axis=0)
    sequence = np.arange(sum(sizes))
    np.random.shuffle(sequence)
    
    X = np.array(X)
    X = X[sequence, :]
    Y = Y[sequence, :]
    np.save("X", X)
    np.save("Y", Y)
    return X, Y

In [ ]:
def load_fromNumpy():
    X = np.load("X.npy")
    Y = np.load("Y.npy")
    return X, Y

In [ ]:
def show_someImages(X, Y):
    j = 1;
    plt.figure(figsize=(10,10))
    for i in np.random.randint(0, Y.shape[0], 9):
        plt.subplot(3, 3, j)
        plt.imshow(X[i],cmap="gray")
        plt.grid(False)
        plt.xlabel('{}'.format(Y[i]))
        j += 1
    plt.show()

In [ ]:
def train_model(X, Y):
#This block creates a shallow NN model and fits the data
#checkpoints are stored in car_training
#model will be saved in current directory

    checkpoint_path = 'car_training/cp-{epoch:04d}.ckpt'
    checkpoint_dir  = os.path.dirname(checkpoint_path)
    cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                    verbose=1,
                                                    save_weights_only=True,
                                                    period=5)
    model = keras.Sequential([
        keras.layers.Flatten(input_shape= (X.shape[1], X.shape[2])),
        keras.layers.Dense(1024, activation=tf.nn.relu),
        keras.layers.Dense(32, activation=tf.nn.relu),
        keras.layers.Dense(4, activation=tf.nn.softmax)
    ])
    model.compile(optimizer='sgd',
                 loss='mean_squared_error',
                 metrics=['accuracy'])
    model.save_weights(checkpoint_path.format(epoch=0))
    model.fit(X, Y, epochs=20, callbacks=[cp_callback])
    return model

In [ ]:
def save_model(model):
    model.save("lane_model.h5")

In [ ]:
def eval_testData(model):
    data = ['forward', 'left', 'right', 'idle']
    X_test = []
    test_sizes = []
    for direction in data:
        address = './' + direction + '_test/*.jpg'
        file_list = glob.glob(address)
        test_sizes.append(len(file_list))
        print('len of {}_test is {}'.format(direction, len(file_list)))
        for image in file_list:
            x = Image.open(image)
            x = np.array(x.convert('L'))
            X_test.append(x)
    X_test = np.array(X_test)
    #labels
    #labels
    y_forward = 0* np.ones((test_sizes[0],1))
    y_left    = 1* np.ones((test_sizes[1],1))
    y_right   = 2* np.ones((test_sizes[2],1))
    y_idle    = 3* np.ones((test_sizes[3],1))
    Y_test = np.concatenate((y_forward, y_left, y_right, y_idle),axis=0)
    test_loss, test_acc = model.evaluate(X_test, Y_test)
    print('Test accuracy: {:05.2f}%'.format(test_acc*100))

In [ ]:
# Use either one:
#X, Y = load_dataSet()
#one-Hot encoding for Y if loaded from images:
#Y = keras.utils.to_categorical(Y, 4)
X, Y = load_fromNumpy()

#Run this to show 9 Random Images
show_someImages(X, Y)

#Train & save the Model
model = train_model(X, Y)
save_model(model)

#Evaluate Model on test data!
eval_testData(model)